# Assignment no.2: 
Implementation of information retrieval system (TREC Collection)

## 💬 Commentary
This solution was supposed to implement similar solution to the one from first assignment 
(https://github.com/adamhospodka/Information_Retrieval_System/blob/main/Implementation_of_information_retrieval_system.ipynb) 
adding the ML capabilities.

However due to time/performance issues I had to downgrade from the additional ML process. It only relies on TF-IDF model.

Unfortunatelly, even this simple solution still scores *0.00 MAP*. I give up.



Steps:

1. List every word from every document using ```multi_buildPairs()``` function
2. Remove duplicates using the ```uniq()``` function
3. Based on these pairs create inverted index using the ```buildInvertedIndex()``` function
4. Use knowledge of inverted index to build frequency index (inv. index with word frequencies) using the ```multi_buildFrequencyIndex()``` function
5. Build index that contains length of each document using the ```buildDocumentsLengthIndex()``` function
6. Prepare Pandas DataFrame with document instances ready to be ranked using the ```buildRankingDf()``` function
7. Initialize the search with ```IRSystem.search()```
8. Clean the given query with the ```cleanQuery()``` function
9. Rank the relevant documents with the ```rank()``` function

## 📚 Imports

In [ ]:
! pip install git+https://gitlab.fi.muni.cz/xstefan3/pv211-utils.git@master | grep '^Successfully'
! pip install pandas
! pip install numpy
! pip install nltk

In [1]:
import math
import time
import pickle
import json
import numpy as np
import pandas as pd

import nltk
from nltk.corpus import stopwords

from tqdm import tqdm
from collections import OrderedDict
from itertools import chain

from pv211_utils.trec.loader import load_documents
from pv211_utils.trec.entities import TrecQueryBase
from pv211_utils.trec.loader import load_queries
from pv211_utils.trec.entities import TrecDocumentBase
from pv211_utils.trec.loader import load_judgements
from pv211_utils.trec.leaderboard import TrecLeaderboard
from pv211_utils.trec.eval import TrecEvaluation
from pv211_utils.trec.irsystem import TrecIRSystemBase


In [ ]:
stemmer = nltk.PorterStemmer()
stopwords = stopwords.words('english')

In [ ]:
nltk.download('punkt')
nltk.download('stopwords')

## 🏗 Classes & Instances loading

#### Documents/Corpus

In [2]:
class Document(TrecDocumentBase):
    """
    A preprocessed TREC collection document
    """

    def __init__(self, document_id: str, body: str):        
        super().__init__(document_id, body)

In [3]:
documents = load_documents(Document, cache_download='/var/tmp/pv211/trec_documents.json.gz')

Computing MD5: /var/tmp/pv211/trec_documents.json.gz
MD5 matches: /var/tmp/pv211/trec_documents.json.gz


In [4]:
CORPUS = documents

#### Queries

In [5]:
class Query(TrecQueryBase):
    """
    A preprocessed TREC collection query
    """    

    def __init__(self, query_id: int, title: str, body: str, narrative: str):

        super().__init__(query_id, title, body, narrative)

In [6]:
train_queries = load_queries(Query, 'train')
validation_queries = load_queries(Query, 'validation')

bigger_train_queries = OrderedDict(chain(train_queries.items(), validation_queries.items()))

#### Judgements

In [7]:
train_judgements = load_judgements(train_queries, documents, 'train')
validation_judgements = load_judgements(validation_queries, documents, 'validation')

bigger_train_judgements = train_judgements | validation_judgements

---

## ⚙️ Preprocessing

#### Tokenizing bodies

In [2]:
def tokenize(document):
    """
    Creates a dictionary with document ids (keys) and list of stemmed, tokenized terms (values).
    Constructed to be compatible with the pool.map() function.
    """
    
    doc_id = document.document_id
    body = document.body
    body = nltk.word_tokenize(body)
    body = [stemmer.stem(token) for token in body]
    
    return {doc_id: body}

In [ ]:
# Parallel processing
with Pool(None) as pool:
    tokenized_bodies = pool.map(tokenize, tqdm(CORPUS.values()) )

In [ ]:
# Reforming
tokenized_bodies_unzipped = {}
for document in tokenized_bodies:
    key = list(document.keys())[0]
    tokenized_bodies_unzipped[key] = document[key]

##### Save and load pickle

In [ ]:
# Save
with open('/data/xhospodk/tokenized_bodies_unzipped.pkl', 'wb') as f:
    pickle.dump(tokenized_bodies_unzipped, f)

In [8]:
# Load
with open('/data/xhospodk/tokenized_bodies_unzipped.pkl', 'rb') as f:
    tokenized_bodies_unzipped = pickle.load(f)

#### Build Pairs

In [ ]:
def multi_buildPairs(document):
    """
    Crawls every document instance and creates a tuples -> (token, document id). 
    It only does it so when token obeys certain conditions.
    """
    
    pairs = []
    punc = '''!(),---[]{};::'"\ <>./?@#$%^&*_␣~\n\n'''
       
    tokens = tokenized_bodies_unzipped[document.document_id]

    token = ""
    doc_id = ""

    for token in tokens:
        if len(token) > 2:
            if token not in stopwords:
                for letter in token:
                    if letter in punc:
                        token = token.replace(letter, "")
                            
                doc_id = document.document_id
                pairs.append((token, doc_id))
    
    return pairs

In [ ]:
# Parallel processing
with Pool() as pool:
    multi_pairs = pool.map(multi_buildPairs, tqdm(CORPUS.values() ))

In [ ]:
# Reforming
multi_pairs_unzipped = []
for array_of_tuples in tqdm(multi_pairs):
    for tuple in array_of_tuples:
        multi_pairs_unzipped.append(tuple)

#### Uniq

In [ ]:
def uniq(sorted_list):
    """
    Reduces the list of pairs only to unique tuples (a term can be only once in a document)
    """
    
    if len(sorted_list) <= 1:
        return sorted_list

    uniq_list = sorted_list[:1]
    previous_value = sorted_list[0]

    for value in sorted_list[1:]:
        if value != previous_value:
            uniq_list.append(value)
            previous_value = value
                
    return uniq_list

In [ ]:
uniq_pairs = uniq(sorted(tqdm(multi_pairs_unzipped), key=lambda x: (x[0].lower(), x[1])))

#### Inverted Index

In [ ]:
def buildInvertedIndex(uniq_pairs):
      """ 
      Creates inverted index using the list of unique pairs (token, document id).
      Returns inverted index (dictionary)
      """

    inverted_index = {}

    for term, document_id in tqdm(multi_pairs_unzipped):
        if term not in inverted_index:
            inverted_index[term] = []

        inverted_index[term].append(document_id)
    
    return inverted_index

In [ ]:
inverted_index = buildInvertedIndex(uniq_pairs)

##### Save and load pickle

In [ ]:
# Save
with open('/data/xhospodk/inverted_index.pkl', 'wb') as f:  # Save
    pickle.dump(inverted_index, f)

In [9]:
# Load
with open('/data/xhospodk/inverted_index.pkl', 'rb') as f:
    inverted_index = pickle.load(f)

#### Frequency Index

In [ ]:
def multi_buildFrequencyIndex(key):
    """ 
    Creates frequency index (inverted index with term frequencies)
    using inverted index as heuristics for crawling.
    Returns frequency index (dictinoary).
    """
  
    local_list = {}
    relevant_documents = inverted_index[key]

    for doc_id in relevant_documents:
      body = tokenized_bodies_unzipped[doc_id]
      frequency = body.count(key)
      local_list[doc_id] = frequency

    return local_list

In [ ]:
# Parallel processing
with Pool(processes=60) as pool:
    frequency_index = pool.map(multi_buildFrequencyIndex, tqdm(inverted_index.keys()) )

##### Save and load pickle

In [ ]:
with open('/data/xhospodk/frequency_index.pkl', 'wb') as f:  # Save
    pickle.dump(frequency_index, f)

In [ ]:
with open('frequency_index.pkl', 'rb') as f:
    frequency_index = pickle.load(f)

#### Documents Length Index

In [10]:
def buildDocumentsLengthIndex():
    """ 
    Creates index containing information about documents lengths.
    Returns documents length index (dictionary).
    """

    documents_length_index = {}

    for key in tqdm(CORPUS.keys()):
        documents_length_index[key] = len(tokenized_bodies_unzipped[key])
    
    return documents_length_index

In [11]:
documents_length_index = buildDocumentsLengthIndex()

100%|██████████| 527890/527890 [00:01<00:00, 480614.34it/s]


## 🧮 Ranking Dataframe

In [20]:
def buildRankingDf():
    """ 
    Prepares dataframe (table) with document instances loaded.
    Ranking and sorting will be happening within this structure once called.
    Returns dataframe.
    """
    
    id_as_list = [key for key, value in tqdm(CORPUS.items())]
    text_as_list = [value.body[:10] for key, value in tqdm(CORPUS.items())]

    df = pd.DataFrame({"ID":  id_as_list, "Values": text_as_list, "Rank": 0, "Matches": "" }) 
    df = df.set_index("ID")

    return df

In [21]:
ranking_df = buildRankingDf()

100%|██████████| 527890/527890 [00:02<00:00, 263076.86it/s]


In [25]:
ranking_df.shape

(527890, 3)

## 🧹 Query Cleaning

In [57]:
def cleanQuery(query):
  """ 
  Cleans passed query's body.
  Tokenization, stemming, stopwords and (words< 1) removed.
  Returns clean query body (array).
  """        

  query = query.body
  query = nltk.word_tokenize(query)
  query = [stemmer.stem(token) for token in query]
  query = [token for token in query if len(token) > 1 if token not in stopwords]

  return query

___

## 💯 Ranking Mechanism

In [134]:
def rank(query):
    """ 
    For every token in query ranks documents +1 for every token appearing in the document
    """

    global ranking_df
    ranking_df["Rank"] = 0.0
    ranking_df["Matches"] = ""

    for term in query:
        if term in inverted_index:
            for relevant_document_id in inverted_index[term]:
                if term not in ranking_df.at[relevant_document_id, "Matches"]:
                    ranking_df.at[relevant_document_id, "Rank"] += 1
                    ranking_df.at[relevant_document_id, "Matches"] +=  " " + term


    ranking_df = ranking_df.sort_values("Rank", ascending = False)
    sorted_documents = ranking_df["Values"].tolist()

    return sorted_documents

## 🚧 Information retrieval system and evaluation

In [135]:
class IRSystem(TrecIRSystemBase):

    def __init__(self, print_matrix):
        self.print_matrix = print_matrix
        random_documents = sorted(set(document for query, document in train_judgements))
        self.random_documents = random_documents

    def search(self, query: Query) -> Iterable[Document]:
        query = cleanQuery(query)
        sorted_documents = rank(query)

        if self.print_matrix == True:
          print("Query: ",query)
          print(ranking_df.head(20))

        return(sorted_documents)

In [136]:
submit_result = False
author_name = "Hospodka, Adam"
system = IRSystem(print_matrix = True)


print('Initializing your system ...', end='', flush=True)
test_queries = load_queries(Query, 'test')
test_judgements = load_judgements(test_queries, documents, 'test')
evaluation = TrecEvaluation(system, test_judgements, TrecLeaderboard(), author_name)
print(end='\r', flush=True)
evaluation.evaluate(tqdm(test_queries.values(), desc="Querying your system, brother", leave=False), submit_result)

Querying your system, brother:   0%|          | 0/50 [00:00<?, ?it/s]

<Query 401 “What language and cultural differences impede the  ...”>
<class '__main__.Query'>
Query:  ['languag', 'cultur', 'differ', 'imped', 'integr', 'foreign', 'minor', 'germani']
                        Values  Rank  \
ID                                     
FBIS4-20472         Policy (Wh   8.0   
FBIS3-42766         Language:    8.0   
FBIS3-36078         Language:    8.0   
FBIS3-30533         Language:    7.0   
FBIS3-41666         Federation   7.0   
FT924-3751          FOR THE tr   7.0   
FBIS3-20098         Language:    7.0   
FBIS3-31672         Language:    7.0   
FBIS4-46969         CSO [Round   7.0   
FBIS3-23561         Article      7.0   
FBIS3-24361         Language:    7.0   
FBIS3-44708         Language:    7.0   
FBIS3-35195         Language:    7.0   
FBIS3-31620         Language:    7.0   
FBIS4-46766         CSO [Dialo   7.0   
FR940602-1-00023  Thursday\n\n   7.0   
FBIS3-19645         Language:    7.0   
FBIS3-7981          Language:    7.0   
FBIS4-40131     

Querying your system, brother:   2%|▏         | 1/50 [00:25<21:04, 25.80s/it]

<Query 402 “What is happening in the field of behavioral genet ...”>
<class '__main__.Query'>
Query:  ['happen', 'field', 'behavior', 'genet', 'studi', 'rel', 'influenc', 'genet', 'environment', 'factor', 'individu', "'s", 'behavior', 'person']
                      Values  Rank  \
ID                                   
FBIS4-20472       Policy (Wh  11.0   
LA100790-0251     Everything  10.0   
FBIS3-60342       the Former  10.0   
FBIS4-68893       CSO [Forew  10.0   
FR940104-0-00034  21 CFR Par  10.0   
FBIS3-37947       Language:   10.0   
FBIS3-24469       Language:   10.0   
LA062590-0042     At a confe  10.0   
LA122489-0017     THE WORLD   10.0   
FR940727-2-00007  National O  10.0   
FBIS3-54460       Language:   10.0   
FBIS3-14832       Language:   10.0   
FBIS3-36078       Language:   10.0   
FBIS3-61193       Language:   10.0   
FBIS4-29          Foreign     10.0   
FBIS3-41666       Federation  10.0   
FBIS3-40450       Language:    9.0   
FBIS3-58841       Language:    9.

Querying your system, brother:   4%|▍         | 2/50 [00:53<21:21, 26.70s/it]

...

<Query 450 “How significant a figure over the years was the la ...”>
<class '__main__.Query'>
Query:  ['signific', 'figur', 'year', 'wa', 'late', 'jordanian', 'king', 'hussein', 'peac', 'middl', 'east']
                     Values  Rank  \
ID                                  
FT944-14204     MONDAY 24\n   9.0   
FT943-13705    18\nMONDAY\n   9.0   
LA033189-0105    During his   9.0   
FBIS4-66185    CSO \n\n       8.0   
FT924-8277       JORDAN'S r   8.0   
FBIS4-67533    CSO \n\n       8.0   
FT943-12422     MONDAY 25\n   8.0   
LA021289-0041    A one-time   8.0   
FT911-1699       A joint in   8.0   
FBIS3-36078      Language:    8.0   
FBIS3-37947      Language:    8.0   
LA082790-0013    As I write   8.0   
LA021989-0037    What a dif   8.0   
FBIS4-36533      BFN [Studi   8.0   
FBIS4-37832      BFN [Inter   8.0   
FT932-12056      WHEN King    8.0   
FT921-7455       Talking of   8.0   
FT944-15625      Just how d   8.0   
LA053190-0100    Arab leade   8.0   
LA040189-0010    Kin

Your system achieved **0.00% MAP score**.

You need at least **10%** to pass. 😢

Try playing with the preprocessing of queries and documents! 💡

Set `submit_result = True` and write your name to the `author_name` variable to submit your result to [the leaderboard](https://docs.google.com/spreadsheets/d/e/2PACX-1vQ33YdFZtGH6g2bDbkD9aLozLdVVGNuP09sRh-F9d_EY9nWntOrLHSyNATFsXw4v9lw3UA3vOzl5l0s/pubhtml). 🏆

The best submissions on the leaderboard will receive *small awards during the semester*, and some *__seriously big__ awards* after the personal check at the end of the competition (2021-05-16). Please be polite, do not spoil the game for the others, and **have fun!** 😉